In [39]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import pandas_profiling

import unidecode

import swifter

import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

import nltk
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
import spacy
nlp_es = spacy.load("es", disable=['parser', 'tagger', 'ner'])
stops_es = stopwords.words("spanish")

import unidecode

import dask
import dask.distributed

from imblearn.over_sampling import SMOTE

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from dask_ml.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from dask_ml.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report

import joblib
from joblib import dump, load

import dask
from dask.distributed import Client

from normalize import normalize

In [111]:
c = Client()
c.cluster

# Español

### 1. Limpieza de texto

In [16]:
df_train_es = pd.read_csv('data/train_es.csv').drop('Unnamed: 0', axis = 1)


In [17]:
%%time
## Español
df_train_es['Text_After_Clean'] = df_train_es.title.swifter.apply(normalize,
                                                             lowercase=True,
                                                             remove_stopwords=True,
                                                             nlp = nlp_es,
                                                             stops = stops_es)




CPU times: user 1h 51min 57s, sys: 3min 10s, total: 1h 55min 7s
Wall time: 2h 6min 17s


In [18]:
df_train_es.sample(5, random_state = 5)

,title,label_quality,language,category,Text_After_Clean
7314493,Mix Relax: Menta + Pino + Romero Madu,unreliable,spanish,ESSENTIAL_OILS,mix relax : menta + pino + romero madu
3239013,Pc Escritorio Cx Slim Intel I7 7700 1tb 8gb Dv...,unreliable,spanish,DESKTOP_COMPUTERS,pc escritorio cx slim intel i tb gb dvdrw cx
2006711,Lapiz Pantalla Capacitiva + Linterna Y Laser+l...,unreliable,spanish,STYLUSES,lapiz pantalla capacitivo + linterna laser+lap...
4828593,Parlante Noga Mix Par Amplificados Inalambrico...,unreliable,spanish,SPEAKERS,parlante noga mix par amplificar inalambricos ...
9985400,Preservativos M Display Mforce + Multio 10 Pac...,unreliable,spanish,CONDOMS,preservativo metro display mforce + multio pac...


In [20]:
df_train_es.shape

(10000000, 5)

In [22]:
df_train_es.columns

Index(['title', 'label_quality', 'language', 'category', 'Text_After_Clean'], dtype='object')

In [ ]:
dask.compute(df_train_es.label_quality.value_counts())

In [23]:
df_train_es.drop('title', axis = 1, inplace = True)


In [3]:
## Store and load train_es dataframe
#df_train_es.to_csv('data/train_es_full.csv')
df_train_es = pd.read_csv('data/train_es_full.csv')

In [4]:
df_train_es.label_quality.value_counts()

unreliable    9509073
reliable       490927
Name: label_quality, dtype: int64

In [5]:
df_train_es_reliable = df_train_es[df_train_es.label_quality == 'reliable']

## Term-frecuency Descriptor

### Stratified subsampling n = 10000

In [21]:
# undersampling con máximo 10**4 elementos de cada clase
undersamp_es = df_train_es_reliable.dropna(subset = ['Text_After_Clean']).groupby('category', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))
undersamp_es.drop(['Unnamed: 0', 'label_quality'], axis = 1, inplace = True)

In [22]:
undersamp_es.shape

(111293, 4)

In [23]:
undersamp_es.sample(5)

,title,language,category,Text_After_Clean
797578,Zapatillas Nike Sb Mod Stefan Janoski Max Neg...,spanish,SNEAKERS,zapatilla nike sb mod stefan janoski max negro...
1821963,"Cuna Graco Nueva! En Caja Cerrada, Sin Uso! Co...",spanish,BABY_PLAYARDS,"cunar graco nuevo ! caja cerrar , usar ! cambi..."
7360168,"Crop Top Bordado En Lentejuelas, Encaje Vestid...",spanish,DRESSES,"crop top bordar lentejuela , encajar vestir fi..."
1594083,Guantes 12oz Box Muay Thai,spanish,MARTIAL_ARTS_AND_BOXING_GLOVES,guante oz box muay thai
5427636,Aplique De Pared Exterior Tortuga Media Caña P...,spanish,WALL_LIGHTS,aplicar pared exterior tortuga medio cana pvc ...


In [24]:
undersamp_es.loc[undersamp_es.Text_After_Clean.isna()]

,title,language,category,Text_After_Clean


Al parecer hay casos en los que `Text_After_Clean` es `NaN` debido a que el título no contiene ninguna de las keywords extraidas, en este caso por ejemplo, aún extrayendo otras keywords del título sería casi imposible obtener información relevante. **Por ahora no tengo una solución para esto.**



-----
Carga del vocabulario con las palabras más relevantes obtenidas con TF-IDF

In [40]:
vocab_es = np.load('vocab_es.npy', allow_pickle = True)

Extracción del descriptor de frecuencias, entrenado sobre df_train_es y usado para generar el descriptor sobre undersample

In [43]:
frecuencies_descriptor_es = CountVectorizer(max_features = 1000000, strip_accents='unicode', max_df = .1, analyzer = 'word',
                                            token_pattern = '\w{3,9}', vocabulary= vocab_es).fit(df_train_es_reliable.Text_After_Clean.dropna())
frecuencies_mat_es = frecuencies_descriptor_es.transform(undersamp_es.Text_After_Clean)


In [131]:
dump(frecuencies_descriptor_es, 'frecuencies_descriptor_es.joblib')
#load('frecuencies_descriptor_es.joblib')

['frecuencies_descriptor_es.joblib']

In [75]:
#dump(frecuencies_mat_es, 'data/frecuencies_mat_es.joblib')
#frec_mat_2 = load('data/frecuencies_mat_es.joblib')

Agregar la columna de lenguaje para crear la matriz de atributos

In [45]:
X = sparse.hstack([frecuencies_mat_es, pd.get_dummies(undersamp_es.language).values]) # Esto tiene todo el descriptor + lenguaje

In [46]:
X.shape

(111293, 12425)

Voy a pasar la columna completa y que sklearn haga el encoding en tiempo de entrenamiento

In [47]:
Y = undersamp_es.category

In [96]:
df_counts = pd.DataFrame(Y.value_counts(), Y.value_counts().index)
df_counts[df_counts.category > 4].index

Index(['CONCERT_TICKETS', 'SOCKS', 'SHORTS', 'MOBILE_DEVICE_CHARGERS',
       'SUITCASES', 'NAIL_DRYERS', 'SOFAS', 'MEMORY_CARDS', 'STOOLS',
       'FOG_LIGHTS',
       ...
       'TEA_INFUSERS', 'VISES', 'MICROWAVE_KEYPADS', 'GAS_DETECTORS',
       'GARDENING_AND_CONSTRUCTION_SHOVELS', 'CAR_INTERCOOLERS',
       'AUTOMOTIVE_ENGINE_CYLINDER_HEADS', 'CORSETS', 'BABY_BOTTLE_WARMERS',
       'CAR_SCANNERS'],
      dtype='object', length=961)

-----
## Oversample

Voy a tener que realizar oversampling para compensar el severo desbalance de clases que hay para algunas clases con frecuencia cercana a 1:


In [97]:
resample_dict = dict(zip(df_counts[df_counts.category > 4].index.values, [200]*len((df_counts[df_counts.category > 4].index.values))))


In [98]:
sm = SMOTE(sampling_strategy = resample_dict, random_state = 11235813, n_jobs = -1, k_neighbors=4)
X_balanced_es, Y_balanced_es = sm.fit_resample(X, Y)

In [99]:
X_balanced_es.shape

(192449, 12425)

No voy a poder remuestrear todas las clases porque aquellas que tienen frecuencia de 1 o 2 significaría dejar al estimador de KNN con un k_neighbors de 1 o 2, lo cual probablemente de malos resultados al momento de sintentizar muestras. Dejé como k_neighbors 4 y se fijan como objetivo aquellas que tienen una frecuencia de 4 o más solamente.

In [108]:

sparse.save_npz('data/X_balanced_es.npz', X_balanced_es)
dump(Y_balanced_es, 'data/Y_balanced_es.joblib')


['data/Y_balanced_es.joblib']

-------


In [81]:
#dump(X, 'data/X_es.joblib')
dump(Y, 'data/Y_es.joblib')
scipy.sparse.save_npz('data/X_es.npz', X)
#scipy.sparse.save_npz('data/Y_es.npz', Y.values)


In [82]:
#X = load('data/X_es.joblib')
Y = load('data/Y_es.joblib')
X = scipy.sparse.load_npz('data/X_es.npz')


KeyboardInterrupt: 

In [ ]:
X

distributed.nanny - WARNING - Restarting worker


In [ ]:
Y

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


La matriz X contiene el descriptor de frecuencia de términos de las palabras de los títulos además de la columna de lenguaje binarizada.

El siguiente paso es entrenar un modelo

# Modelamiento

In [114]:
from sklearn.multiclass import OneVsRestClassifier

In [121]:
%%time
#with joblib.parallel_backend("dask"):
clf = OneVsRestClassifier(RandomForestClassifier(random_state = 11235813), n_jobs = -1).fit(X_balanced_es, Y_balanced_es)

CPU times: user 34min 25s, sys: 5min 19s, total: 39min 44s
Wall time: 9h 11min 55s


In [130]:
clf.estimator

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=11235813,
                       verbose=0, warm_start=False)

In [128]:
dump(clf.estimator, 'data/rf_balanced_estimator.joblib')

['data/rf_balanced_estimator.joblib']

In [51]:
%%time

grid_params_svc = {'C':np.logspace(-3,3,10)}
grid_svc = GridSearchCV(RandomForestClassifier(random_state = 11235813, gamma = 'auto'), param_grid = grid_params_svc, n_jobs = -1, cv=5)

#with joblib.parallel_backend("dask"):
grid_svc.fit(X, undersamp_es.category)

/Users/ignacio/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

Nota: Si las clases con muy pocos elementos impiden entrenar bien el clasificador, luego del subsampling del df va a haber que hacer oversampling con smote

In [ ]:
df_test_es = pd.read_csv('data/test_es')
df_test_es.shape

In [ ]:

%%time
## Español
df_test_es['Text_After_Clean'] = df_test_es.title.swifter.apply(normalize,
                                                             lowercase=True,
                                                             remove_stopwords=True,
                                                             nlp = nlp_es,
                                                             stops = stops_es)




In [ ]:
%%time
X_test_frec = frecuencies_descriptor_es.transform(df_test_es.Text_After_Clean)
X_test = sparse.hstack([X_test_frec, pd.get_dummies(df_test_es.language).values]) # Esto tiene todo el descriptor + lenguaje

In [ ]:
preds = clf.predict(X_test)
print(classification_report(df_test_es.category, preds))